In [1]:
import cv2
import numpy as np
import tensorflow as tf
import os
import pywt
from scipy.fftpack import dct, idct
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

def build_generator():
    inputs = tf.keras.layers.Input(shape=(128, 128, 1))
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    model = tf.keras.Model(inputs, x, name="Generator")
    return model

# Load the generator model
generator = build_generator()
weights_path = "./model_weights/generator_epoch_4900.weights.h5"

if os.path.exists(weights_path):
    generator.load_weights(weights_path)
    print("✅ Generator weights loaded successfully!")
else:
    raise FileNotFoundError(f"❌ Weights file not found at {weights_path}")

# Define paths
INPUT_VIDEO_PATH = "./input1.mp4"
OUTPUT_VIDEO_PATH = "./output_watermarked.mp4"
EXTRACTED_WATERMARK_PATH = "./extracted_watermarks"

os.makedirs(EXTRACTED_WATERMARK_PATH, exist_ok=True)

# Open the input video
cap = cv2.VideoCapture(INPUT_VIDEO_PATH)

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Open the output video writer
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))

frame_count = 0
psnr_values = []
ssim_values = []
watermark_ssim_values = []  # To store SSIM between generated and extracted watermarks

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  

    frame = frame.astype(np.float32) / 255.0  

    # Split the frame into RGB channels
    R, G, B = cv2.split(frame)

    # Convert frame to grayscale and resize for the generator
    gray_frame = cv2.cvtColor((frame * 255).astype(np.uint8), cv2.COLOR_BGR2GRAY)
    resized_frame = cv2.resize(gray_frame, (128, 128)) / 255.0  

    # Generate the watermark using the generator
    input_frame = np.expand_dims(resized_frame, axis=[0, -1])
    watermark = generator.predict(input_frame)[0, :, :, 0]

    # Apply DWT to embed the watermark
    def apply_dwt_watermark(channel):
        LL, (LH, HL, HH) = pywt.dwt2(channel, 'haar')
        watermark_resized = cv2.resize(watermark, (HH.shape[1], HH.shape[0]))
        HH_watermarked = HH + 0.005 * watermark_resized
        return pywt.idwt2((LL, (LH, HL, HH_watermarked)), 'haar')

    R_dwt = apply_dwt_watermark(R)
    G_dwt = apply_dwt_watermark(G)
    B_dwt = apply_dwt_watermark(B)

    # Apply DCT to embed the watermark
    def apply_dct_watermark(channel):
        dct_channel = dct(dct(channel.T, norm='ortho').T, norm='ortho')

        dct_height, dct_width = dct_channel.shape

        h_start, h_end = int(0.1 * dct_height), int(0.3 * dct_height)
        w_start, w_end = int(0.1 * dct_width), int(0.3 * dct_width)

        watermark_resized = cv2.resize(watermark, (w_end - w_start, h_end - h_start))

        dct_channel[h_start:h_end, w_start:w_end] += 0.01 * watermark_resized

        idct_channel = idct(idct(dct_channel.T, norm='ortho').T, norm='ortho')
        return idct_channel

    R_dct = apply_dct_watermark(R_dwt)
    G_dct = apply_dct_watermark(G_dwt)
    B_dct = apply_dct_watermark(B_dwt)

    # Merge the watermarked channels
    watermarked_frame = cv2.merge((R_dct, G_dct, B_dct))
    watermarked_frame = np.clip(watermarked_frame * 255, 0, 255).astype(np.uint8)
    out.write(watermarked_frame)

    # Calculate PSNR and SSIM between original and watermarked frames
    original_frame = (frame * 255).astype(np.uint8)
    psnr_value = psnr(original_frame, watermarked_frame, data_range=255)
    ssim_value = ssim(original_frame, watermarked_frame, data_range=255, win_size=3, channel_axis=-1)

    psnr_values.append(psnr_value)
    ssim_values.append(ssim_value)

    # Save the extracted watermark
    extracted_watermark_path = os.path.join(EXTRACTED_WATERMARK_PATH, f"frame_{frame_count:04d}.png")
    cv2.imwrite(extracted_watermark_path, (watermark * 255).astype(np.uint8))

    # Load the extracted watermark for comparison
    extracted_watermark = cv2.imread(extracted_watermark_path, cv2.IMREAD_GRAYSCALE)
    if extracted_watermark is None:
        raise ValueError(f"❌ Unable to load extracted watermark from {extracted_watermark_path}")

    # Normalize the extracted watermark to [0, 1]
    extracted_watermark = extracted_watermark.astype(np.float32) / 255.0

    # Calculate SSIM between the generated watermark and the extracted watermark
    watermark_ssim = ssim(watermark, extracted_watermark, data_range=1.0)
    watermark_ssim_values.append(watermark_ssim)

    frame_count += 1

# Release video capture and writer
cap.release()
out.release()

# Calculate average metrics
avg_psnr = np.mean(psnr_values)
avg_ssim = np.mean(ssim_values)
avg_watermark_ssim = np.mean(watermark_ssim_values)

# Print results
print(f"✅ Watermarked video saved: {OUTPUT_VIDEO_PATH}")
print(f"✅ Extracted watermarks saved in: {EXTRACTED_WATERMARK_PATH}")
print(f"📊 Average PSNR: {avg_psnr:.2f} dB (Higher is better)")
print(f"📊 Average SSIM (Frame Similarity): {avg_ssim:.4f} (Closer to 1 is better)")
print(f"📊 Average SSIM (Watermark Similarity): {avg_watermark_ssim:.4f} (Closer to 1 is better)")
print(f"🎯 Model Accuracy (Based on Watermark SSIM): {avg_watermark_ssim * 100:.2f}%")

✅ Generator weights loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 526ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━